## First download all of the necessary data

---

Press "Shift + Enter" to run each cell sequentially. Alternatively, you can press "Cmd/Ctrl + F9" to run all cells and then scroll down to bottom cell.

In [ ]:
cd

In [ ]:
# Download Pre-requisites needed for running the e2e model
%cd content
%mkdir model_data
import urllib.request
url = 'https://dl.fbaipublicfiles.com/pythia/data/answers_vqa.txt'
filename = '../model_data/answers_vqa.txt'
urllib.request.urlretrieve(url, filename)
url = 'https://dl.fbaipublicfiles.com/pythia/data/vocabulary_100k.txt'
filename = '../model_data/vocabulary_100k.txt'
urllib.request.urlretrieve(url, filename)
url = 'https://dl.fbaipublicfiles.com/pythia/data/vocabulary_100k.txt'
filename = '../model_data/vocabulary_100k.txt'
urllib.request.urlretrieve(url, filename)
url = 'https://dl.fbaipublicfiles.com/pythia/detectron_model/detectron_model.pth'
filename = '../model_data/detectron_model.pth'
urllib.request.urlretrieve(url, filename)
url = 'https://dl.fbaipublicfiles.com/pythia/pretrained_models/vqa2/pythia_train_val.pth'
filename = '../model_data/pythia.pth'
urllib.request.urlretrieve(url, filename)
url = 'https://dl.fbaipublicfiles.com/pythia/pretrained_models/vqa2/pythia_train_val.yml'
filename = '../model_data/pythia.yaml'
urllib.request.urlretrieve(url, filename)
url = 'https://dl.fbaipublicfiles.com/pythia/detectron_model/detectron_model.yaml'
filename = '../model_data/detectron_model.yaml'
urllib.request.urlretrieve(url, filename)
url = 'https://dl.fbaipublicfiles.com/pythia/data/detectron_weights.tar.gz'
filename = '../model_data/detectron_weights.tar.gz'
urllib.request.urlretrieve(url, filename)




## Now, install some particular dependencies

In [ ]:
!tar xf ~/content/model_data/detectron_weights.tar.gz

In [ ]:
!pip install torchvision 

In [ ]:
# Install dependencies
!pip install yacs cython matplotlib
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI
!pip3 install --upgrade Pillow
!pip install gTTS
!pip install pygame
!pip install -q torchaudio omegaconf soundfile
!pip install ffmpeg
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio
!pip install .\PyAudio-0.2.11-cp38-cp38-win_amd64.whl

In [ ]:
!pip install .\PyAudio-0.2.11-cp39-cp39-win_amd64.whl

In [ ]:
cd ..

## Install MMF now

In [ ]:
%cd ~/content/
!git clone https://github.com/facebookresearch/mmf.git mmf
%cd ~/content/mmf
!pip install -e .
import sys
sys.path.append("~/content/mmf")

In [ ]:
!pip install lmdb

In [ ]:
%cd ~/content/
!git clone https://github.com/facebookresearch/mmf.git mmf
%cd ~/content/mmf
!pip install --editable . --user
import sys
sys.path.append("~/content/mmf")

In [ ]:
pip install editdistance

In [ ]:
cd ..

## Install maskrcnn-benchmark now

In [ ]:
pip install numpy --upgrade --user

In [ ]:
!pip install ninja

In [ ]:
%cd ~

In [ ]:
!conda install libpython m2w64-toolchain -c msys2

In [ ]:
!pip uninstall apex
%cd apex
!python setup.py install --cuda_ext --cpp_ext

In [1]:
# Install maskrcnn-benchmark to extract detectron features
import sys
%cd ~/content
!git clone https://gitlab.com/meetshah1995/vqa-maskrcnn-benchmark.git
%cd ~/content/vqa-maskrcnn-benchmark
# Compile custom layers and build mask-rcnn backbone
!python setup.py build develop
sys.path.append('~/content/vqa-maskrcnn-benchmark')

C:\Users\53444\content
C:\Users\53444\content\vqa-maskrcnn-benchmark


fatal: destination path 'vqa-maskrcnn-benchmark' already exists and is not an empty directory.


running build
running build_py
running build_ext


C:\Users\53444\anaconda3\lib\site-packages\torch\utils\cpp_extension.py:316: UserWarning: Error checking compiler version for g++: Command 'g++' returned non-zero exit status 1.
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')
error: [WinError 2] 系统找不到指定的文件。
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "C:\Users\53444\anaconda3\lib\site-packages\colorama\ansitowin32.py", line 59, in closed
    return stream.closed
ValueError: underlying buffer has been detached


In [ ]:
import sys
sys.path.append('~/content/vqa-maskrcnn-benchmark/maskrcnn_benchmark')

In [ ]:
#cat imports.py
!pip install sentencepiece
!pip install numpy
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning

In [ ]:
!pip install opencv-python
!pip install torchvision

In [ ]:
!conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

In [ ]:
!pip install ninja

In [ ]:
cd content

In [ ]:
!pip install intel-openmp

## Demo

The class handles everything from feature extraction, token extraction and predicting the answer

In [2]:
%cd ~/content/apex
import yaml
import cv2
import torch
import requests
import numpy as np
import gc
import torch.nn.functional as F
import pandas as pd


import torchvision.models as models
import torchvision.transforms as transforms

from PIL import Image
from IPython.display import display, HTML, clear_output
from ipywidgets import widgets, Layout
from io import BytesIO
from argparse import Namespace


from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.layers import nms
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.structures.image_list import to_image_list
from maskrcnn_benchmark.utils.model_serialization import load_state_dict


from mmf.datasets.processors.processors import VocabProcessor, VQAAnswerProcessor
from mmf.models.pythia import Pythia
from mmf.common.registry import registry
from mmf.common.sample import Sample, SampleList
from mmf.utils.env import setup_imports
from mmf.utils.configuration import Configuration

setup_imports()

class MMFDemo:
  TARGET_IMAGE_SIZE = [448, 448]
  CHANNEL_MEAN = [0.485, 0.456, 0.406]
  CHANNEL_STD = [0.229, 0.224, 0.225]
  
  def __init__(self):
    self._init_processors()
    self.pythia_model = self._build_pythia_model()
    self.detection_model = self._build_detection_model()
    self.resnet_model = self._build_resnet_model()
    
  def _init_processors(self):
    args = Namespace()
    args.opts = [
        "config=projects/pythia/configs/vqa2/defaults.yaml",
        "datasets=vqa2",
        "model=pythia",
        "evaluation.predict=True"
    ]
    args.config_override = None

    configuration = Configuration(args=args)
    
    config = self.config = configuration.config
    vqa_config = config.dataset_config.vqa2
    text_processor_config = vqa_config.processors.text_processor
    answer_processor_config = vqa_config.processors.answer_processor
    
    text_processor_config.params.vocab.vocab_file = "~/content/model_data/vocabulary_100k.txt"
    answer_processor_config.params.vocab_file = "~/content/model_data/answers_vqa.txt"
    # Add preprocessor as that will needed when we are getting questions from user
    self.text_processor = VocabProcessor(text_processor_config.params)
    self.answer_processor = VQAAnswerProcessor(answer_processor_config.params)

    registry.register("vqa2_text_processor", self.text_processor)
    registry.register("vqa2_answer_processor", self.answer_processor)
    registry.register("vqa2_num_final_outputs", 
                      self.answer_processor.get_vocab_size())
    
  def _build_pythia_model(self):
    state_dict = torch.load('C:/Users/53444/content/model_data/pythia.pth')
    model_config = self.config.model_config.pythia
    model_config.model_data_dir = "~/content/model_data/"
    model = Pythia(model_config)
    model.build()
    model.init_losses()
    
    if list(state_dict.keys())[0].startswith('module') and \
       not hasattr(model, 'module'):
      state_dict = self._multi_gpu_state_to_single(state_dict)
          
    model.load_state_dict(state_dict, strict=False)
    model.to("cuda")
    model.eval()
    
    return model
  
  def _build_resnet_model(self):
    self.data_transforms = transforms.Compose([
        transforms.Resize(self.TARGET_IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(self.CHANNEL_MEAN, self.CHANNEL_STD),
    ])
    resnet152 = models.resnet152(pretrained=True)
    resnet152.eval()
    modules = list(resnet152.children())[:-2]
    self.resnet152_model = torch.nn.Sequential(*modules)
    self.resnet152_model.to("cuda")
  
  def _multi_gpu_state_to_single(self, state_dict):
    new_sd = {}
    for k, v in state_dict.items():
        if not k.startswith('module.'):
            raise TypeError("Not a multiple GPU state of dict")
        k1 = k[7:]
        new_sd[k1] = v
    return new_sd
  
  def predict(self, url, question):
    with torch.no_grad():
      detectron_features = self.get_detectron_features(url)
      resnet_features = self.get_resnet_features(url)

      sample = Sample()

      processed_text = self.text_processor({"text": question})
      sample.text = processed_text["text"]
      sample.text_len = len(processed_text["tokens"])

      sample.image_feature_0 = detectron_features
      sample.image_info_0 = Sample({
          "max_features": torch.tensor(100, dtype=torch.long)
      })

      sample.image_feature_1 = resnet_features

      sample_list = SampleList([sample])
      sample_list = sample_list.to("cuda")

      scores = self.pythia_model(sample_list)["scores"]
      scores = torch.nn.functional.softmax(scores, dim=1)
      actual, indices = scores.topk(5, dim=1)

      top_indices = indices[0]
      top_scores = actual[0]

      probs = []
      answers = []

      for idx, score in enumerate(top_scores):
        probs.append(score.item())
        answers.append(
            self.answer_processor.idx2word(top_indices[idx].item())
        )
    
    gc.collect()
    torch.cuda.empty_cache()
    
    return probs, answers
    
  
  def _build_detection_model(self):

      cfg.merge_from_file('~/content/model_data/detectron_model.yaml')
      cfg.freeze()

      model = build_detection_model(cfg)
      checkpoint = torch.load('~/content/model_data/detectron_model.pth', 
                              map_location=torch.device("cpu"))

      load_state_dict(model, checkpoint.pop("model"))

      model.to("cuda")
      model.eval()
      return model
  
  def get_actual_image(self, image_path):
      if image_path.startswith('http'):
          path = requests.get(image_path, stream=True).raw
      else:
          path = image_path
      
      return path

  def _image_transform(self, image_path):
      path = self.get_actual_image(image_path)

      img = Image.open(path)
      im = np.array(img).astype(np.float32)
      im = im[:, :, ::-1]
      im -= np.array([102.9801, 115.9465, 122.7717])
      im_shape = im.shape
      im_size_min = np.min(im_shape[0:2])
      im_size_max = np.max(im_shape[0:2])
      im_scale = float(800) / float(im_size_min)
      # Prevent the biggest axis from being more than max_size
      if np.round(im_scale * im_size_max) > 1333:
           im_scale = float(1333) / float(im_size_max)
      im = cv2.resize(
           im,
           None,
           None,
           fx=im_scale,
           fy=im_scale,
           interpolation=cv2.INTER_LINEAR
       )
      img = torch.from_numpy(im).permute(2, 0, 1)
      return img, im_scale


  def _process_feature_extraction(self, output,
                                 im_scales,
                                 feat_name='fc6',
                                 conf_thresh=0.2):
      batch_size = len(output[0]["proposals"])
      n_boxes_per_image = [len(_) for _ in output[0]["proposals"]]
      score_list = output[0]["scores"].split(n_boxes_per_image)
      score_list = [torch.nn.functional.softmax(x, -1) for x in score_list]
      feats = output[0][feat_name].split(n_boxes_per_image)
      cur_device = score_list[0].device

      feat_list = []

      for i in range(batch_size):
          dets = output[0]["proposals"][i].bbox / im_scales[i]
          scores = score_list[i]

          max_conf = torch.zeros((scores.shape[0])).to(cur_device)

          for cls_ind in range(1, scores.shape[1]):
              cls_scores = scores[:, cls_ind]
              keep = nms(dets, cls_scores, 0.5)
              max_conf[keep] = torch.where(cls_scores[keep] > max_conf[keep],
                                           cls_scores[keep],
                                           max_conf[keep])

          keep_boxes = torch.argsort(max_conf, descending=True)[:100]
          feat_list.append(feats[i][keep_boxes])
      return feat_list

  def masked_unk_softmax(self, x, dim, mask_idx):
      x1 = F.softmax(x, dim=dim)
      x1[:, mask_idx] = 0
      x1_sum = torch.sum(x1, dim=1, keepdim=True)
      y = x1 / x1_sum
      return y
   
  def get_resnet_features(self, image_path):
      path = self.get_actual_image(image_path)
      img = Image.open(path).convert("RGB")
      img_transform = self.data_transforms(img)
      
      if img_transform.shape[0] == 1:
        img_transform = img_transform.expand(3, -1, -1)
      img_transform = img_transform.unsqueeze(0).to("cuda")
      
      features = self.resnet152_model(img_transform).permute(0, 2, 3, 1)
      features = features.view(196, 2048)
      return features
    
  def get_detectron_features(self, image_path):
      im, im_scale = self._image_transform(image_path)
      img_tensor, im_scales = [im], [im_scale]
      current_img_list = to_image_list(img_tensor, size_divisible=32)
      current_img_list = current_img_list.to('cuda')
      with torch.no_grad():
          output = self.detection_model(current_img_list)
      feat_list = self._process_feature_extraction(output, im_scales, 
                                                  'fc6', 0.2)
      return feat_list[0]
    

C:\Users\53444\content\apex


ModuleNotFoundError: No module named 'maskrcnn_benchmark'

### If the command below fails with 'CUDNN_EXECUTION_FAILED', try rerunning the cell

In [ ]:
cd ..

In [ ]:
demo = MMFDemo()

In [ ]:
cd content

## Use the text fields below to ask a question on an image

Image URL can be any http/https URL. We show top 5 predictions from MMF. Confidence shows how confident MMF model was about a particular prediction.

In [ ]:
from gtts import gTTS
import pygame
import os
import torch
import zipfile
import torchaudio
from glob import glob
from pydub import AudioSegment
import pyaudio
import wave

FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 10
WAVE_OUTPUT_FILENAME = "recording.wav"

def init_widgets(url, question):
  image_text = widgets.Text(
    description="Image URL", layout=Layout(minwidth="70%")
  )
  question_text = widgets.Text(
      description="Question", layout=Layout(minwidth="70%")
  )

  image_text.value = url
  question_text.value = question
  submit_button = widgets.Button(description="Ask MMF!")

  display(image_text)
  display(question_text)
  display(submit_button)

  submit_button.on_click(lambda b: on_button_click(
      b, image_text, question_text
  ))
  
  return image_text, question_text
  
def on_button_click(b, image_text, question_text):
  #recording part
  audio = pyaudio.PyAudio()
  stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)
  print("recording...")
  frames = []
  for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
  print("finished recording")
  stream.stop_stream()
  stream.close()
  audio.terminate()
  waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
  waveFile.setnchannels(CHANNELS)
  waveFile.setsampwidth(audio.get_sample_size(FORMAT))
  waveFile.setframerate(RATE)
  waveFile.writeframes(b''.join(frames))
  waveFile.close()

  #stt part
  s = ""

  #wav_audio = AudioSegment.from_file("Recording.m4a", format="m4a")
  #wav_audio.export("recording.wav", format="wav")    

  device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU

  model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                        model='silero_stt',
                                        language='en', # also available 'de', 'es'
                                        device=device)
  (read_batch, split_into_batches,
  read_audio, prepare_model_input) = utils  # see function signature for details

  test_files = glob('recording.wav')
  batches = split_into_batches(test_files, batch_size=10)
  input = prepare_model_input(read_batch(batches[0]),
                              device=device)

  output = model(input)
  for example in output:
      s = decoder(example.cpu())

  #output part
  clear_output()
  image_path = demo.get_actual_image(image_text.value)
  image = Image.open(image_path)
  
  #scores, predictions = demo.predict(image_text.value, question_text.value)
  scores, predictions = demo.predict(image_text.value, s)
  scores = [score * 100 for score in scores]
  df = pd.DataFrame({
      "Prediction": predictions,
      "Confidence": scores
  })
  language = 'en'
  myobj = gTTS(text=predictions[0], lang=language, slow=False)
  myobj.save("output.mp3")
  pygame.mixer.init()
  pygame.mixer.music.load('output.mp3')
  pygame.mixer.music.play()
  init_widgets(image_text.value, question_text.value)
  display(image)
  display(HTML(df.to_html()))
 



image_text, question_text = init_widgets(
    "VizWiz_val_00028000.jpg", 
    s
)